In [0]:
import tensorflow as tf 
import numpy as np 
import random


class MLP:
    def __init__(self,size_hidden,dim_feature,n_class):
        self.dim_feature=dim_feature
        self.size_hidden=size_hidden
        self.n_class=n_class
        self.X=None
        self.Y_true=None
    
    def build_graph(self):
        tf.compat.v1.disable_eager_execution()
        self.X=tf.compat.v1.placeholder(tf.float32,shape=[None,self.dim_feature])
        self.Y_true=tf.compat.v1.placeholder(tf.int32,shape=[None,])

        weight1= tf.Variable(name="weight_input_hidden",
                            initial_value=tf.random.normal([self.dim_feature,self.size_hidden]))
        bias1= tf.Variable(name="bias_hidden",
                            initial_value=tf.random.normal([self.size_hidden]))

        weight2=tf.Variable(name="weight_hidden_output",
                            initial_value=tf.random.normal([self.size_hidden,self.n_class]))
        bias2=tf.Variable(name="bias_ouput",
                            initial_value=tf.random.normal([self.n_class]))

        hidden=tf.matmul(self.X,weight1)+bias1
        hidden=tf.sigmoid(hidden)
        logits=tf.matmul(hidden,weight2)+bias2

        labels_one_hot= tf.one_hot(indices=self.Y_true,depth=self.n_class,dtype=tf.int32)
        loss=tf.nn.softmax_cross_entropy_with_logits(labels=labels_one_hot,logits=logits)
        loss= tf.reduce_mean(loss)
        prob= tf.nn.softmax(logits)
        pre_labels=tf.argmax(prob,axis=1)
        
        return pre_labels,loss

    def trainer(self,lr,loss):
        train= tf.compat.v1.train.AdamOptimizer(lr).minimize(loss)
        return train



class DataReader:
    def __init__(self,path,size_batch,dim_feature):
        def convert_to_vector(doc,dim_feature):
            vector=np.zeros(dim_feature)
            feas=doc.split()
            for fea in feas:
                v=fea.split(':')
                vector[int(v[0])]=float(v[1])
            return vector

        with open(path,'r') as f:
            d_line=f.read().splitlines()
        self.x=[]
        self.y=[]
        self.size_batch=size_batch
        for d in d_line:
            fea=d.split('<<>>')
            self.y.append(fea[0])
            fea_vector=convert_to_vector(fea[-1],dim_feature)
            self.x.append(fea_vector)
        self.n_doc=len(d_line)
        self.n_batch=int(np.ceil(self.n_doc/size_batch))
        rd=random.sample(range(self.n_doc),self.n_doc)
        self.x=np.array(self.x)[rd]
        self.y=np.array(self.y)[rd]
        self.batch_id=0
        self.epoch=0
    
    def next_batch(self):
        if(self.batch_id>=self.n_batch):
            self.batch_id=0
            self.epoch+=1
            rd=random.sample(range(self.n_doc),self.n_doc)
            self.x=self.x[rd]
            self.y=self.y[rd]
        
        start=self.batch_id*self.size_batch
        end=start+self.size_batch
        self.batch_id+=1
        batch_x=self.x[start:end]
        batch_y=self.y[start:end]

        return batch_x,batch_y

def save_paras(name,value,path):
    value_form=[]
    if(len(value.shape)==1):
        value_form=[str(number) for number in value]
    else:
        for row in range(len(value)):
            value_form.append(' '.join([str(number) for number in value[row]]))
    with open(path+'/'+name+'.txt','w') as f:
        f.write('\n'.join(value_form))

def restore_paras(path_in,name):
    with open(path_in+'/'+name+'.txt','r') as f:
        data=f.read().splitlines()
    values=[]
    if(len(data)==1):
        value=[float(number) for number in data.split()]
    else:
        for d in data:
            values.append([float(number) for number in d.split()])
    
    return values


In [2]:
if __name__ == "__main__":
    '''
    with open("",'r') as f:
        dim_feature=len(f.read().splitlines())
    '''
    path_train="/content/drive/My Drive/Project2/Datasets/train_tfidf_df=3"
    path_test="/content/drive/My Drive/Project2/Datasets/test_tfidf"
    dim_feature=20167
    mlp=MLP(50,dim_feature,20)
    pre_label,loss=mlp.build_graph()
    trainer=mlp.trainer(0.01,loss=loss)

    init = tf.compat.v1.global_variables_initializer()
    with tf.compat.v1.Session() as sess:
        data_train=DataReader(path_train,50,20167)
        data_test=DataReader(path_test,50,20167)
        it,MAX_epoch=0,100
        sess.run(init)
        while it<MAX_epoch:
            batch_train_x,batch_train_y=data_train.next_batch()
            pre_val,loss_val,_= sess.run(
                [pre_label,loss,trainer],
                feed_dict={
                    mlp.X:batch_train_x,
                    mlp.Y_true:batch_train_y
                }
            )
            if(it<data_train.epoch):
                print("epoch {}, =loss ={}".format(data_train.epoch,loss_val))
            it=data_train.epoch
        
        trainable_variables=tf.compat.v1.trainable_variables()
        for variable in trainable_variables:
            save_paras(
                name=variable.name,
                value=variable.eval(),
                path='/content/drive/My Drive/Lab/paras'
            )
        

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
epoch 1, =loss =0.5492036938667297
epoch 2, =loss =0.20984907448291779
epoch 3, =loss =0.03416093811392784
epoch 4, =loss =0.009996453300118446
epoch 5, =loss =0.007882528007030487
epoch 6, =loss =0.006103637162595987
epoch 7, =loss =0.009011822752654552
epoch 8, =loss =0.008451980538666248
epoch 9, =loss =0.0032189900521188974
epoch 10, =loss =0.010175035335123539
epoch 11, =loss =0.0022902474738657475
epoch 12, =loss =0.0016660152468830347
epoch 13, =loss =0.001060745446011424
epoch 14, =loss =0.0007377241272479296
epoch 15, =loss =0.0010383084882050753
epoch 16, =loss =0.0011014753254130483
epoch 17, =loss =0.000725111342035234
epoch 18, =loss =0.0007288033375516534
epoch 19, =loss =0.000783333380240947
epoch 20, =loss =0.0009273638715967536
epoch 21, =loss =0.000646027794573456
epoch 22, =loss =0.0003212660667486489
epoch 23, =loss =0.0002680794568732381
epoch 24, =loss =0.0003647305420599878
epoch 25,

In [6]:
with tf.compat.v1.Session() as sess:
    data_test=DataReader(path_test,50,20167)
    sess.run(init)
    for variable in trainable_variables:
        saved_value=restore_paras('/content/drive/My Drive/Lab/paras',variable.name)
        saved_value=np.squeeze(saved_value)
        assign_op=variable.assign(saved_value)
        sess.run(assign_op)
    num_true_pre=0
    while True:
        batch_test_x,batch_test_y=data_test.next_batch()
        pre_val= sess.run(
            pre_label,
            feed_dict={
                mlp.X:batch_test_x,
                mlp.Y_true:batch_test_y              
            }
        )
        pre_val=np.array(pre_val,dtype=int)
        batch_test_y=np.array(batch_test_y,dtype=int)
        num_true_pre+=sum(pre_val==batch_test_y)
        if(data_test.epoch==1):
            break
    print(float(num_true_pre/len(data_test.y)))

0.7619490175252257
